# datasets 폴더 내 llm.zip 파일 압축 풀기

In [2]:
import zipfile

In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/LLM/datasets

/content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/LLM/datasets


In [ ]:
with zipfile.ZipFile("llm.zip", 'r') as zip_ref:
    zip_ref.extractall()

# 반복문을 이용한 데이터 수집

In [ ]:
### /datasets/TL_span_extraction.json 파일

In [ ]:
!pip install datasets==2.21.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x8

In [6]:
!pip show huggingface_hub

Name: huggingface-hub
Version: 0.30.2
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, diffusers, peft, sentence-transformers, timm, tokenizers, transformers


In [ ]:
import requests
import json
import pandas as pd
from datasets import Dataset
import huggingface_hub

### JSON 형태의 행정 문서 대상 기계독해 데이터 (출처: AI허브 -> "금융, 법률 문서 기계독해 데이터")

In [ ]:
file_path = "/content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/LLM/datasets/TL_span_extraction.json"
try:
    # 파일 열고 내용 읽기
    with open(file_path, "r", encoding="utf-8-sig") as f:  # 적절한 인코딩 설정
        original_data = json.load(f)
        print(original_data)

except Exception as e:
    print(e)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 원본데이터의 키 확인하기
original_data.keys()

dict_keys(['Dataset', 'data'])

In [ ]:
len( original_data["data"] )

45418

In [ ]:
datas = original_data["data"]
datas

[{'doc_id': 'TNL041227',
  'doc_title': '신 미·일방위협력지침의 주요내용 및 쟁점과 한국의 대응과제',
  'doc_source': '국회입법조사처',
  'doc_published': 20150603,
  'created': '20221219143503',
  'doc_class': {'class': '법률/사회과학/법학일반', 'code': '연구보고서'},
  'paragraphs': [{'context_id': '2356574',
    'context': '1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.',
    'tbs': None,
    'qas': [{'qa_type': 1,
      'question_id': '6379347',
      'question': '처음으로 미·일방위협력지침이 정해진 건 언제야',
      'answer': {'source': '2356574',
       'text': '1978년',
       'answer_start': 109,
       'answer_end': 113,
       'cell_text': None,
       'cell_coordinates': None,
       'clue_start': None,
       'clue_text': None,
       'opti

In [ ]:
alpaca_data = []

columnList = []

In [ ]:
# 수집해야 하는 데이터
targetClass = "법률/사회과학/노동·법률정책"

In [ ]:
for data in datas:
    # 하나의 데이터 셋 내 주요 파라그래프 키 확인하기
    data.keys()

    if(data["doc_class"]["class"] == targetClass):

        for eachParagraph in data["paragraphs"]:
            # 딕셔너리 구조의 키 중 주요 정보 키값으로 가져오기
            qasDatas = eachParagraph["qas"]

            for qasData in qasDatas:
                columnList = []
                # 질문
                columnList.append(qasData["question"])

                # 정답
                columnList.append(qasData["answer"]["text"])

                # input
                columnList.append("")

                # class
                # columnList.append(data["doc_class"]["class"])

                alpaca_data.append(columnList)


### instrunction : 질문
### output       : 정답
### input        : 없음
Df = pd.DataFrame(alpaca_data)
Df.columns = ["instruction", "output", "input"]
Df

,instruction,output,input
0,3차 감염자까지 나오고 있어서 신속한 대처가 요구되는 신종 감염병이 뭐야,메르스,
1,새로운 종류의 감염병으로 3차 감염자까지 발생되어 빠른 대처방안이 필요한 건 뭐야,메르스,
2,외국에서 들어오는 전염병을 가장 먼저 파악할 수 있는 곳은 어디야,입국장,
3,어디에서 외국 발 전염병의 최초 확인이 가능해,입국장,
4,의심 환자를 옮길 때 항공기 소독시행을 지시하고 이를 질병관리본부에 알려야 하는 사...,검역관,
...,...,...,...
5451,뭐가 발달하게 될 새로운 교통시스템의 시대를 자율주행자동차가 견인할 수 있어,온라인 교통서비스 시장,
5452,자율주행자동차로 인해 교통안전이 증진하는 것과 함께 기대되는 게 뭐야,교통시스템의 효율성 증대,
5453,교통안전의 증대와 어떤 효과를 자율주행자동차 도입으로 기대할 수 있어,교통시스템의 효율성 증대,
5454,자율주행 관련 제도를 만드는 데에 있어 견해를 청취해야 하는 대상은 누구야,이해당사자,


In [ ]:
# DataFrame에서 하나의 값만 보기
Df.loc[0,["instruction"]].values

array(['3차 감염자까지 나오고 있어서 신속한 대처가 요구되는 신종 감염병이 뭐야'], dtype=object)

In [ ]:
# prompt: 허깅페이스 로그인 코드생성

import os
from huggingface_hub import notebook_login

# Hugging Face Hub에 로그인합니다.
notebook_login()


In [ ]:
dataset = Dataset.from_pandas(Df)

In [ ]:
dataset.push_to_hub("limjh12/law")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/limjh12/law/commit/f477a6db766f258f82fe349a0c85e87b2e629dee', commit_message='Upload dataset', commit_description='', oid='f477a6db766f258f82fe349a0c85e87b2e629dee', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/limjh12/law', endpoint='https://huggingface.co', repo_type='dataset', repo_id='limjh12/law'), pr_revision=None, pr_num=None)